In [23]:
import re
import glob
from collections import Counter

In [5]:
REL_DATA_PATH = '../raw_gantt_chart_data/output/'
data_condition_dir = glob.glob(REL_DATA_PATH + '*')

pattern = r'\-\d{3,}\S+.csv'

In [7]:
for data_dir in data_condition_dir:
    condition_type = data_dir.split('/')[-1]
    wf_type_list = [re.sub(pattern, '', x.split('/')[-1]) for x in glob.glob(data_dir + '/*')]

In [27]:
wf_counts = Counter(wf_type_list)

In [29]:
wf_counts

Counter({'nowcast-clustering-8-casa_nowcast-wf': 60,
         'nowcast-clustering-16-casa_nowcast-wf': 60,
         '1000genome': 50,
         'wind-noclustering-casa_wind_wf': 60,
         'wind-clustering-casa_wind_wf': 60})